# Get Change in Estimate for Incremented Numerosity for Scalar Networks

Select the model, and make sure that the batch_size in model_settings.py is 9000 (the number of sets times the number of numerosities). Make sure analysis_estimation_nds.py imports FF_estimation_scalar.py.

In [1]:
import numpy as np
import plotly.graph_objs as go
import importlib
import analysis_estimation_nds
importlib.reload(analysis_estimation_nds)
import create_data_po
importlib.reload(create_data_po)
from jupyterthemes import jtplot
jtplot.style()
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

['/home/sychen23/.local/lib/python3.5/site-packages/ipykernel_launcher.py', '-f', 'true', 'true', 'true', 'true', 'true', 'model_runs/-f/classify_log.csv', 'model_runs/-f/classifymodel_0.ckpt', 'model_runs/-f/classifymodel_', 'model_runs/-f/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
analysis_estimation_nds.py
analysis_estimation_nds.py


In [2]:
models = [("New_CAA_const_fN_scalar", "Const Avg Area, $f(n)$ Constant"),
          ("New_CAA_decrs_fN_scalar", "Const Avg Area, $f(n) \propto 1/n^2$"),
          ("New_DAA_const_fN_scalar", "Decr Avg Area, $f(n)$ Constant"),
          ("New_DAA_decrs_fN_scalar", "Decr Avg Area, $f(n) \propto 1/n^2$")]
model = models[3]
model_name, model_figure_title = model
num_runs = 10
iter_list = [0, 200, 400, 1600, 6400, 25600, 102400, 204800, 409600, 819200, 1000000, 1228800, 1638400, 2000000, 3000000]#, 4000000, 5000000, 6000000]
train_max_blobs = 15
test_max_blobs = 9
train_min_blobs = test_min_blobs = 1

num_iters = len(iter_list)
num_sets = 1000
train_output_size = train_max_blobs - train_min_blobs + 1
test_output_size = test_max_blobs - test_min_blobs + 1
test_condition = "po_inc"
data_directory = "data/" + model_name + "/" + test_condition + "/"

import os
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
    
scalar = True

In [3]:
def get_values_one_run(path, iteration):
    """Get scalar value estimates for one run at one iteration."""

    imgs_data = analysis_estimation_nds.classify_imgs_po_sets(iteration, True, path=path, scalar=scalar) # new_imgs = True
    unrounded_values = np.zeros([num_sets, train_output_size])
    rounded_values = np.zeros([num_sets, train_output_size])
    
    for i in range(num_sets):
        for idx, data in enumerate(imgs_data[i*test_output_size:i*test_output_size+test_output_size]):
            nb = int(data["label"][0])
            classification = data["classifications"][0][0]
            unrounded_values[i, nb-1] = classification
            rounded_values[i, nb-1] = round(classification)
    return unrounded_values, rounded_values

In [4]:
# Test out one iteration
path = 'model_runs/' + model_name + str(1)
imgs_data = analysis_estimation_nds.classify_imgs_po_sets(0, True, path=path, scalar=scalar)
print(np.shape(imgs_data[0]['label']))
print([imgs_data[i]['label'][0] for i in range(0,9)]) # 0 to 8 make up first set, numerosities from 9 to 1.
print(imgs_data[800]['classifications'][0][0]) # scalar output

INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_0.ckpt
(1,)
[9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]
-1.7351615


In [5]:
i = 2
[x['label'] for x in imgs_data[i*test_output_size:i*test_output_size+test_output_size]]

[array([9.]),
 array([8.]),
 array([7.]),
 array([6.]),
 array([5.]),
 array([4.]),
 array([3.]),
 array([2.]),
 array([1.])]

In [ ]:
unrounded_values_all_runs = np.zeros([num_runs, num_iters, num_sets, train_output_size]) 
rounded_values_all_runs = np.zeros([num_runs, num_iters, num_sets, train_output_size])
for run in range(num_runs):
    path = 'model_runs/' + model_name + str(run + 1)
    for i, iteration in enumerate(iter_list):
        unrounded_values_all_runs[run, i], rounded_values_all_runs[run, i] = get_values_one_run(path, iteration)

INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_0.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_200.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_1600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_6400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_25600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_102400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_204800.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/classifymodel_409600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar1/cl

INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_25600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_102400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_204800.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_409600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_819200.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_1000000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_1228800.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_1638400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_decrs_fN_scalar6/classifymodel_2000000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_DAA_

In [ ]:
np.save(data_directory + "unrounded_values_all_runs_set", unrounded_values_all_runs)
np.save(data_directory + "rounded_values_all_set", rounded_values_all_runs)

In [ ]:
print("Unrounded")
unrounded_values_all_runs = np.load(data_directory + 'unrounded_values_all_runs_set.npy') # runs, iters, sets, inputs, outputs
d_unrounded = np.diff(unrounded_values_all_runs[:, -1, :, :9], axis=2) # last iteration, avg over sets, up to numerosity 9, mean
print('mean of d_unrounded: %s' % np.mean(np.mean(d_unrounded, axis=1), axis=0))
print('sd of d_unrounded: %s' % np.mean(np.std(d_unrounded, axis=1), axis=0))